# DATA DOWNLOAD AND EXPLORATION

This first notebook will cover the **installation** of the dataset to be used. This dataset is provided by _SoccerNet_ and contains over 3.000 multi-view videos for training, validation and testing, and provides 273 actions for the challenge.

We start by installing the __SoccerNet-v2__, wich will provide us the necessary tools for the download of the dataset.


In [ ]:
pip install SoccerNet --upgrade

We will use the _SoccerNetDownloader_ tool for installing the dataset. To do this we declare the path were we want the dataset to be stored. 

For this download we need to adquire a **license of use** through this [link](https://docs.google.com/forms/d/e/1FAIpQLSfYFqjZNm4IgwGnyJXDPk2Ko_lZcbVtYX73w5lf6din5nxfmA/viewform).

In [9]:
import SoccerNet

from SoccerNet.Downloader import SoccerNetDownloader
mySoccerNetDownloader = SoccerNetDownloader(LocalDirectory="../data")
psswrd = "s0cc3rn3t"
mySoccerNetDownloader.password = psswrd

In this dataset we will find:

- train: Standard training data for models, with complete annotations.
- valid: Validation data to evaluate performance during development.
- test: Test data with non-public labels (typically used for competitions).
- challenge: A subset of data specific to official competition or specific challenges, such as Multi-View Foul Recognition.

We need the dataset related withe *Multi-view Foul Recognition* challenge, known as __mvfouls__. Also we will download only videos in 720p.

**IMPORTANT: by executing the following code the installation will start, it can last a long time, so be sure of executing it.**

In [2]:
mySoccerNetDownloader.downloadDataTask(task="mvfouls", split=["train","valid","test","challenge"], password = psswrd, version = "720p")

The dataset will be downloaded in _.zip_ files, which must be **unzipped**.

The dataset consists of **3,901 actions**. Each action is composed of at least two videos showing the live action and at least one replay.

The dataset is divided into:
- Training set (2916 actions).
- Validation set (411 actions).
- Test set (301 actions).
- Challenge set (273 actions without the annotations).

We will then use _OpenCV_ to check that the videos and tags have been downloaded correctly.

In [2]:
import cv2
import os

train_dataset_path = "../data/mvfouls/train_720p"

training_videos_path = os.path.join(train_dataset_path)
example_video = os.path.join(training_videos_path, "action_1/clip_0.mp4")

def visualize_video(video_path, num_frames=100):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    
    if not cap.isOpened():
        print(f"No se pudo abrir el video: {video_path}")
        return

    while cap.isOpened() and frame_count < num_frames:
        ret, frame = cap.read()
        if not ret:
            print("Fin del video o error al leer el frame.")
            break
        
        cv2.imshow('Video Frame', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):  # Press 'q' to exit
            break
        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

visualize_video(example_video)

We will also verify that the tags are in the correct format.

In [3]:
import json

labels_path = os.path.join(train_dataset_path, "annotations.json")

with open(labels_path, "r") as f:
    data = json.load(f)

print(f"Total de acciones etiquetadas: {data['Number of actions']}")
for action_index, action_data in data['Actions'].items():
    print(f"Índice de la acción: {action_index}")
    print(f"Detalles de la acción: {action_data}")
    break


Total de acciones etiquetadas: 2916
Índice de la acción: 0
Detalles de la acción: {'UrlLocal': 'england_epl\\2014-2015\\2015-02-21 - 18-00 Chelsea 1 - 1 Burnley', 'Offence': 'Offence', 'Contact': 'With contact', 'Bodypart': 'Upper body', 'Upper body part': 'Use of shoulder', 'Action class': 'Challenge', 'Severity': '1.0', 'Multiple fouls': '', 'Try to play': '', 'Touch ball': '', 'Handball': 'No handball', 'Handball offence': '', 'Clips': [{'Url': 'Dataset/Train/action_0/clip_0', 'Camera type': 'Main camera center', 'Timestamp': 1730826, 'Replay speed': 1.0}, {'Url': 'Dataset/Train/action_0/clip_1', 'Camera type': 'Close-up player or field referee', 'Timestamp': 1744173, 'Replay speed': 1.8}]}


We will extract the information of three actions.

In [5]:
actions = data['Actions']

count = 3

for action_id, action in actions.items():
    print(f"Acción número {action_id}:")
    print(f"  > Ofensa: {action['Offence']}")
    print(f"  > Parte del cuerpo: {action['Bodypart']}")
    print(f"  > Tipo de acción: {action['Action class']}")
    print(f"  > Severidad: {action['Severity']}")
    
    for clip in action['Clips']:
        clip_url = clip['Url']
        camera_type = clip['Camera type']
        timestamp = clip['Timestamp']
        print(f"    - Clip - {camera_type}: {clip_url} @ {timestamp}ms")
    count -= 1
    if count == 0:
        break


Acción número 0:
  > Ofensa: Offence
  > Parte del cuerpo: Upper body
  > Tipo de acción: Challenge
  > Severidad: 1.0
    - Clip - Main camera center: Dataset/Train/action_0/clip_0 @ 1730826ms
    - Clip - Close-up player or field referee: Dataset/Train/action_0/clip_1 @ 1744173ms
Acción número 1:
  > Ofensa: Offence
  > Parte del cuerpo: Under body
  > Tipo de acción: Tackling
  > Severidad: 3.0
    - Clip - Main camera center: Dataset/Train/action_1/clip_0 @ 2400217ms
    - Clip - Close-up player or field referee: Dataset/Train/action_1/clip_1 @ 2415695ms
Acción número 2:
  > Ofensa: Offence
  > Parte del cuerpo: Under body
  > Tipo de acción: Standing tackling
  > Severidad: 3.0
    - Clip - Main camera center: Dataset/Train/action_2/clip_0 @ 206821ms
    - Clip - Close-up player or field referee: Dataset/Train/action_2/clip_1 @ 230429ms
    - Clip - Close-up player or field referee: Dataset/Train/action_2/clip_2 @ 238257ms


In this fragment we will show both clips of an action currently.

In [14]:
import cv2
import os

train_dataset_path = "../data/mvfouls/train_720p"

clip_0_path = os.path.join(train_dataset_path, "action_1/clip_0.mp4")
clip_1_path = os.path.join(train_dataset_path, "action_1/clip_1.mp4")


def visualize_clips_synchronized(clip_1_path, clip_2_path, timestamp_1=0, timestamp_2=0, num_frames=100):
    cap1 = cv2.VideoCapture(clip_1_path)
    cap2 = cv2.VideoCapture(clip_2_path)
    
    if not cap1.isOpened() or not cap2.isOpened():
        print(f"Error al abrir los videos: {clip_1_path}, {clip_2_path}")
        return

    fps1 = cap1.get(cv2.CAP_PROP_FPS)
    fps2 = cap2.get(cv2.CAP_PROP_FPS)

    start_frame_1 = int(timestamp_1 / 1000 * fps1)
    start_frame_2 = int(timestamp_2 / 1000 * fps2)

    cap1.set(cv2.CAP_PROP_POS_FRAMES, start_frame_1)
    cap2.set(cv2.CAP_PROP_POS_FRAMES, start_frame_2)

    frame_count = 0
    while frame_count < num_frames:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        if not ret1 or not ret2:
            print("Fin de uno de los videos o error al leer el frame.")
            break

        combined_frame = cv2.hconcat([frame1, frame2])
        cv2.imshow('Clips Sincronizados', combined_frame)

        if cv2.waitKey(int(1000 / max(fps1, fps2))) & 0xFF == ord('q'):  # Press 'q' to exit
            break

        frame_count += 1

    cap1.release()
    cap2.release()
    cv2.destroyAllWindows()

visualize_clips_synchronized(clip_0_path, clip_1_path, timestamp_1=0, timestamp_2=1000)

In future notebooks we will study the dataset in more detail.